In [1]:
import torch
torch.manual_seed(1337)

# Training hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
eval_iters = 200

# Model hyperparameters
from gpt_config import GPTConfig
config = GPTConfig(
block_size = 8, # what is the maximum context length for predictions?
device = 'cuda' if torch.cuda.is_available() else 'cpu',
n_embd = 64,
n_head = 4,
n_layer = 4,
dropout = 0.0
)

In [2]:
# Our vocabulary is determined by our dataset. We've already download the tiny shakespeare dataset:
# !wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
# read it in to inspect it
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()
print("length of dataset in characters: ", len(text))

length of dataset in characters:  1115389


In [3]:
# Based on the text corpus, we'll build a vocabulary.
# The vocabulary will translate each character into an integer.

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
config.vocab_size = len(chars)
print(f"The corpus contains a total of {config.vocab_size} unique characters: {''.join(chars)}")

# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Now every string can be encoded as a sequence of integers.
print(encode("hello world"))
print(decode(encode("hello world")))

The corpus contains a total of 65 unique characters: 
 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
[46, 43, 50, 50, 53, 1, 61, 53, 56, 50, 42]
hello world


In [4]:
# let's now get the text dataset ready for training
data = torch.tensor(encode(text), dtype=torch.long)
print(f"Full dataset is of size: {data.shape}")

# Let's now split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

print("We're going to train in blocks, and each block gets a growing input that eventually includes the entire block.")
print(f"For example, the first block is: {train_data[:config.block_size]}")

x = train_data[:config.block_size]
y = train_data[1:config.block_size+1]
for t in range(config.block_size):
    context = x[:t+1]
    target = y[t]
    print(f"--->For this block, when input is {context} the target is: {target}")

Full dataset is of size: torch.Size([1115389])
We're going to train in blocks, and each block gets a growing input that eventually includes the entire block.
For example, the first block is: tensor([18, 47, 56, 57, 58,  1, 15, 47])
--->For this block, when input is tensor([18]) the target is: 47
--->For this block, when input is tensor([18, 47]) the target is: 56
--->For this block, when input is tensor([18, 47, 56]) the target is: 57
--->For this block, when input is tensor([18, 47, 56, 57]) the target is: 58
--->For this block, when input is tensor([18, 47, 56, 57, 58]) the target is: 1
--->For this block, when input is tensor([18, 47, 56, 57, 58,  1]) the target is: 15
--->For this block, when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target is: 47
--->For this block, when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target is: 58


In [5]:
# Data / Training Helper functions
# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - config.block_size, (batch_size,))
    x = torch.stack([data[i:i+config.block_size] for i in ix])
    y = torch.stack([data[i+1:i+config.block_size+1] for i in ix])
    x, y = x.to(config.device), y.to(config.device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [6]:
from gpt_zero import GPT
#from gpt2 import GPT
model = GPT(config)
m = model.to(config.device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=config.device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))

0.208961 M parameters
step 0: train loss 4.3422, val loss 4.3382
step 100: train loss 2.6847, val loss 2.6979
step 200: train loss 2.4974, val loss 2.5099
step 300: train loss 2.4194, val loss 2.4496
step 400: train loss 2.3626, val loss 2.3679
step 500: train loss 2.3054, val loss 2.3258
step 600: train loss 2.2860, val loss 2.2949
step 700: train loss 2.2826, val loss 2.2993
step 800: train loss 2.2276, val loss 2.2624
step 900: train loss 2.2075, val loss 2.2358
step 1000: train loss 2.2172, val loss 2.2121
step 1100: train loss 2.1702, val loss 2.2006
step 1200: train loss 2.1630, val loss 2.1893
step 1300: train loss 2.1463, val loss 2.1952
step 1400: train loss 2.1294, val loss 2.1568
step 1500: train loss 2.1102, val loss 2.1628
step 1600: train loss 2.1125, val loss 2.1389
step 1700: train loss 2.1068, val loss 2.1554
step 1800: train loss 2.0849, val loss 2.1466
step 1900: train loss 2.0732, val loss 2.1052
step 2000: train loss 2.0538, val loss 2.1465
step 2100: train loss 2.